# Navigation

This notebook was provided by Udacity as example.

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

[![](https://trello-attachments.s3.amazonaws.com/5d93d97e715aba7c97fde8f7/5e9e457961d16e8a352b67c9/da4327f262b337b51a85826ca5e5cf6f/image.png)](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893)

---

# Index

- [1. Setup the Environment](#1.-Setup-the-Environment)
- [2. Start the Environment](#2.-Start-the-Environment)
- [3. Examine the State and Action Spaces](#3.-Examine-the-State-and-Action-Spaces)
- [4. Develop the Agent to collect bananas](#4.-Develop-the-Agent-to-collect-bananas)
 - [4.1 Define trainer function](#4.1-Define-trainer-function)
 - [4.2 Training agent on environment](#4.2.-Training-agent-on-environment)
 - [4.3 Plot agent losses along the training session](#4.2-Plot-agent-losses-along-the-training-session)
 - [4.4 Save weights](#4.4-Save-weights)
 - [4.5 Load weights](#4.5-Load-weights)
- [5. Test Agent on Environment](#5.-Test-Agent-on-Environment)

### 1. Setup the Environment

<img align="left" width="150" src="https://www.nclouds.com/img/services/toolkit/sagemaker.png"/>

This notebook was developed on AWS SageMaker.

The kernel used is **conda_python3**

To setup this environment on SageMaker you need to run the next 2 cells.

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip -q install ./python

### 2. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
import os
import time
import json
import warnings
import numpy as np

from tqdm import tqdm
from agent import Agent
from datetime import datetime
from collections import deque
import matplotlib.pyplot as plt
from unityagents import UnityEnvironment
from sklearn.model_selection import ParameterGrid

warnings.simplefilter("ignore", UserWarning)
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Next, we will start the environment!  **_Before running the code cell below_**, change the `env_path` variable to match the location of the Unity environment that you downloaded.

- **Linux** (x86_64): `"/Banana_Linux/Banana.x86_64"`
- **Linux** (x86_64, headless): `"/Banana_Linux_NoVis/Banana.x86_64"`

If your system isn't in the list, please, download the environment from here.

The `file_name` parameter needs to be something like that:

```
env = UnityEnvironment(file_name="Banana_Linux_NoVis/Banana.x86_64")
```

In [ ]:
env_path = "Banana_Linux_NoVis/Banana.x86_64"
#env_path = "Banana_Windows_x86_64/Banana.exe"

env = UnityEnvironment(file_name=env_path)

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 3. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

## 4. Define helper functions to Agent trainment

Here we will create two functions that will be very helpful to train the agent.

First one is `dqn_trainer()` that will train the agent in the environment.

Last one is `GridSearch()` that will create all possible combinations from parameter's arrays.
For each time we will save the results on a json log file, it's very helpful to split training time if needed.

In [ ]:
def dqn_trainer(agent, n_episodes=100, print_range=10, eps_start=1.0, eps_end=0.01, eps_decay=0.995, early_stop=13, verbose=False):
    """Deep Q-Learning trainer.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        print_range (int): range to print partials results
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
        early_stop (int): Stop training when achieve a defined score respecting 10 min n_episodes.
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=print_range)  # last 100 scores
    scores_mean = []
    eps = eps_start                    # initialize epsilon
    for i in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        
        while True:
            action = agent.act(state, eps)
            try:
                env_info = env.step(action)[brain_name]
            except:
                print(env.step(action))
                break
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0] 
            agent.step(state, action, reward, next_state, done, i)
            state = next_state
            score += reward
            if done:
                break
                
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        if verbose:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i, np.mean(scores_window)), end="")
            if i % print_range == 0:
                print('\rEpisode {}\tAverage Score: {:.2f}'.format(i, np.mean(scores_window)))
            
        if np.mean(scores_window) >= early_stop and i > 10:
            if verbose:
                print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i, np.mean(scores_window)))
            break
            
    return scores, i, np.mean(scores_window)

def GridSearch(param_grid, n_episodes=200, log_path='gridsearch_logs.json', verbose=False):
    """GridSearch for agent trainer.
    
    Params
    ======
        param_grid (dict): All parameters available for Agent class
        n_episodes (int): The number of episodes that each agent will be trained.
        log_path (str): The JSON file to save parameters.
        verbose (bool): print some information on console
    """
    if not isinstance(log_path, str) or ".json" not in log_path:
        raise ValueError('log_path need to be a string with ".json" in the end.')
        
    try:
        with open(log_path, 'r') as outfile:
            param_grid = json.load(outfile)
        if verbose:
            print("param_grid was imported from file: {}".format(log_path))
    except FileNotFoundError as err:
        param_grid = list(ParameterGrid(param_grid))
        param_grid = [{"params": each} for each in param_grid]
        with open(log_path, 'w') as outfile:
            json.dump(param_grid, outfile)
        if verbose:
            print("No log file available. We created ParameterGrid and save it on file: {}".format(log_path))
    if verbose:
        print("The param_grid will result in {} fits.".format(len(param_grid)))
        time.sleep(0.3)
               
    with tqdm(total=len(param_grid)) as pbar:
        for i in range(len(param_grid)):
            if param_grid[i].get("avg_score") == None: 
                start_time = datetime.now()
                env_info = env.reset(train_mode=True)[brain_name]
                agent = Agent(state_size=param_grid[i]["params"]["state_size"],
                              action_size=param_grid[i]["params"]["action_size"],
                              seed=param_grid[i]["params"]["seed"],
                              nb_hidden=param_grid[i]["params"]["nb_hidden"],
                              learning_rate=param_grid[i]["params"]["learning_rate"],
                              memory_size=param_grid[i]["params"]["memory_size"],
                              prioritized_memory=param_grid[i]["params"]["prioritized_memory"],
                              batch_size=param_grid[i]["params"]["batch_size"],
                              gamma=param_grid[i]["params"]["gamma"],
                              tau=param_grid[i]["params"]["tau"],
                              small_eps=param_grid[i]["params"]["small_eps"],
                              update_every=param_grid[i]["params"]["update_every"])

                score, episodes, last_avg_score = dqn_trainer(agent, n_episodes=n_episodes, verbose=False)
                training_time = datetime.now() - start_time
                
                param_grid[i]["avg_score"] = last_avg_score
                param_grid[i]["training_time"] = str(training_time).split('.')[0] 
                param_grid[i]["score_history"] = score
                param_grid[i]["episodes"] = episodes
                with open(log_path, 'w') as outfile:
                    json.dump(param_grid, outfile)
            pbar.update(1)
            
    best_params = {}
    best_params["episodes"] = 999 # random best_params
    
    for each in param_grid:
        if each["episodes"] < best_params["episodes"]:
            best_params = each
        
    return best_params

### 5. Develop the Agent to collect bananas

In the next code cells, we will train the Agent to work on environment.

### 5.1 Test parameters with GridSearch

First of all, we will apply the `GridSearch()` funtion to find candidates for best parameters. In this first step, our parameters will very different of each other.

In the second part, we will try to refine our parameters a little bit more.

In [ ]:
params = {
    "state_size": [state_size],
    "action_size": [action_size],
    "seed": [199],
    "nb_hidden": [(256, 128), (128, 128), (64, 64)],
    "learning_rate": [1e-5, 1e-4, 1e-3],
    "memory_size": [int(1e5), int(1e4)],
    "prioritized_memory": [False, True],
    "batch_size": [64, 32],
    "gamma": [0.99, 0.95, 0.9, 0.87],
    "tau": [1e-2, 1e-3, 1e-4],
    "small_eps": [1e-2, 1e-3, 1e-4],
    "update_every": [4]
}

In [ ]:
%%time 
best_params = GridSearch(params, n_episodes=200, verbose=True)

### 5.2. Training agent with best parameters

In [ ]:
%%time

env_info = env.reset(train_mode=True)[brain_name]

agent = Agent(state_size, action_size, seed=199, nb_hidden=(256, 128), learning_rate=3e-4,
              memory_size=int(1e5), prioritized_memory=False, batch_size=64,
              gamma=0.9, tau=1e-3, small_eps=1e-2, update_every=4)

scores, episodes, last_avg_score = dqn_trainer(agent, n_episodes=100, verbose=True)

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Average Score')
plt.xlabel('Episode #')
plt.show()

### 5.3 Plot agent losses along the training session

In [ ]:
plt.plot(agent.losses)
plt.title("Losses")
plt.show()

### 5.4 Save weights

In [ ]:
model_path = "model.pt"
agent.save_model(model_path)

### 5.5 Load weights

In [ ]:
model_path = "model.pt"
agent.load_model(model_path)

### 6. Test Agent on Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance.

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]
state = env_info.vector_observations[0]
score = 0
eps = 0.05
while True:
    action = agent.act(state, eps)
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0]
    done = env_info.local_done[0]
    score += reward
    state = next_state
    if done:
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
if False:
    env.close()